In [57]:
import pandas as pd

from itertools import chain

from tfob import TFOb, BHSA, DSS

### Important functions

In [58]:
def clean(g_cons):
    return g_cons.replace("_", " ").replace("׳", "").replace("'", "")



def find_verb_ref(verb):
    if verb.source == "BHSA":
        book = verb.book[0]
        chapter = verb.chapter[0]
        verse_num = verb.verse[0]
    else:
        book = verb.book[0]    
        chapter = verb.chapter[0]
        verse_num = verb.verse[0]
    return [book, chapter, verse_num]



def is_sign_unc(verse):
    """If a verse contains a missing or uncertain sign, returns True. Else, returns False."""
    
    unc_types = ['missing', 'unc']
    verse_sign_types = []
    
    for sign in verse.to_signs:
        if sign.type[0] in unc_types:
            sign_type = 1
        else:
            sign_type = 0
            
        verse_sign_types.append(sign_type)
    
    return verse_sign_types



def is_lex_identical(verb_dss):
    """
    Checks if the verses (i.e. BHSA versus DSS) are identical on the lexeme level.
    """
    ref_verb = find_verb_ref(verb_dss)
    
    scroll = verb_dss.to_scrolls.scroll[0]
    dss_lex = TFOb.section(ref_verb, DSS, scroll).to_words.lex
    bhsa_lex = TFOb.section(ref_verb, BHSA).to_words.lex
    
    if dss_lex[-1] == "":
        dss_lex.pop()
    
    return [clean(lex) for lex in bhsa_lex] == [clean(lex) for lex in dss_lex]

In [59]:
def find_bhsa_verb(verb_dss):
    """
    Checks if a verb occurring in DSS also occurs in BHSA (same book, chapter, verse, lexeme).
    Else, returns None.
    """
    
    # Get book chapter verse info from a DSS verb
    ref_dss = find_verb_ref(verb_dss)

    # Get the corresponding BHSA verse
    verse_bhsa = TFOb.section(ref_dss, BHSA).to_words
    verb_bhsa = verse_bhsa.filter(lex=verb_dss.lex[0])
    
    # If repetition of verb in same verse: TODO
    if len(verb_bhsa) > 1:
        return # TODO
        scroll = verb_dss.to_scrolls.scroll[0]
        verse_dss = TFOb.section(ref_dss, DSS, scroll)
        print("Verse BHSA:", verse_bhsa)
        print("Verse DSS:", verse_dss)
        
    if verb_bhsa:
        return verb_bhsa

    
    
def find_clause(verb):
    """Find the complement of a verb. If no match, returns None"""
    if verb.source.name == "BHSA":
        clause = verb.to_clauses.to_clauses
        return clause
    
    # if the verb is not BHSA, it's DSS
    verb_bhsa = find_bhsa_verb(verb)

    # Check if verses are identical  
    if verb_bhsa and is_lex_identical(verb): # TODO
        scroll = verb.to_scrolls.scroll[0]
        verse_dss = TFOb.section(find_verb_ref(verb), DSS, scroll).to_words
        clause_bhsa = find_clause(verb_bhsa)
        
    
        first_word_id = clause_bhsa.to_words.ids[0]
        last_word_id = clause_bhsa.to_words.ids[-1]

        verse_ids = clause_bhsa.to_verses.to_words.ids

        first_word_index = verse_ids.index(first_word_id)
        last_word_index = verse_ids.index(last_word_id)
            
        
        return verse_dss[first_word_index:last_word_index + 1]
    

    
def find_complements(verb):
    """Find the complement of a verb. If no match, returns None"""
    if verb.source.name == "BHSA":
        complements = verb.to_clauses.to_phrases.filter(function="Cmpl")
        return complements
    
    # if the verb is not BHSA, it's DSS
    verb_bhsa = find_bhsa_verb(verb)

    # Check if verses are identical  
    if verb_bhsa and is_lex_identical(verb): # TODO
        scroll = verb.to_scrolls.scroll[0]
        verse_dss = TFOb.section(find_verb_ref(verb), DSS, scroll).to_words
        complements_bhsa = find_complements(verb_bhsa)
        
        complements_dss = []
    
        for complement_bhsa in complements_bhsa:
            first_word_id = complement_bhsa.to_words.ids[0]
            last_word_id = complement_bhsa.to_words.ids[-1]
            
            verse_ids = complement_bhsa.to_verses.to_words.ids
            
            first_word_index = verse_ids.index(first_word_id)
            last_word_index = verse_ids.index(last_word_id)
            
            complements_dss.append(verse_dss[first_word_index:last_word_index + 1])
        
        return complements_dss


def find_prepositions(verb):
    """Find the preposition in the complement of a verb. If no match, returns None"""
    
    complements = find_complements(verb)
    
    if verb.source.name == "BHSA":
        prepositions = []
        for complement in complements:
            prepositions.append(complement.to_words.filter(sp="prep"))
        return prepositions
    
    # if the verb is not BHSA, it's DSS
    verb_bhsa = find_bhsa_verb(verb)

    # Check if verses are identical  
    if verb_bhsa and is_lex_identical(verb): # TODO
        scroll = verb.to_scrolls.scroll[0]
        verse_dss = TFOb.section(find_verb_ref(verb), DSS, scroll).to_words
        preps_bhsa = find_prepositions(verb_bhsa)
        
        preps_dss = []
    
        for prep_bhsa in preps_bhsa:
            verse_ids = prep_bhsa.to_verses.to_words.ids
            prep_dss_ids = []
            
            for word in prep_bhsa:
                word_id = word.ids[0]
                word_index = verse_ids.index(word_id)
                prep_dss_ids.append(verse_dss.ids[word_index])
            
            preps_dss.append(TFOb(prep_dss_ids, DSS))
        
        return preps_dss    
    

    
def find_subject(verb):
    """Find the subject of a verb. If no match, returns None"""
    if verb.source.name == "BHSA":
        subjects = verb.to_clauses.to_phrases.filter(function="Subj")
        assert len(subjects) <= 1
        return subjects
    
    # if the verb is not BHSA, it's DSS
    verb_bhsa = find_bhsa_verb(verb)

    # Check if verses are identical  
    if verb_bhsa and is_lex_identical(verb): # TODO
        scroll = verb.to_scrolls.scroll[0]
        verse_dss = TFOb.section(find_verb_ref(verb), DSS, scroll).to_words
        subject_bhsa = find_subject(verb_bhsa)
        
        if not subject_bhsa:
            return ""

        first_word_id = subject_bhsa.to_words.ids[0]
        last_word_id = subject_bhsa.to_words.ids[-1]

        verse_ids = subject_bhsa.to_verses.to_words.ids

        first_word_index = verse_ids.index(first_word_id)
        last_word_index = verse_ids.index(last_word_id)
            
        return verse_dss[first_word_index:last_word_index + 1]

### Motion verbs and scrolls

In [60]:
# Adding the missing motion verbs

motion_verbs = ["HWH=[", "PF<[", "CWR=["]


In [61]:
verbs_1q8 = TFOb.all("scroll", DSS).filter(scroll="1Q8").to_words.filter_in(lex=motion_verbs)
verbs_1q8

<none_0 "">

In [62]:
verbs_1qisaa = TFOb.all("scroll", DSS).filter(scroll="1Qisaa").to_words.filter_in(lex=motion_verbs)
verbs_1qisaa

<word_2 ">PF<H TCRJ">

In [63]:
verb1 = TFOb(1918021, DSS)
print(verb1)

cmpl1 = find_complements(verb1)
print(cmpl1)

prep1 = find_prepositions(verb1)
print(prep1)


HWYJTJ
[<word_2 "M J<QWB">]
[<word_1 "M">]


In [64]:
verbs_bhsa = TFOb.all("word", BHSA).filter(book="Isaiah").filter_in(lex=motion_verbs)
verbs_bhsa

<word_2 ">PF<H TCRJ">

In [65]:
verb = verbs_bhsa[0]
print(verb)
cmpl = find_complements(verb)
print(cmpl)
prep = find_prepositions(verb)
print(prep[0].lex)

>PF<H
BH
['B']


### Functions checker

In [66]:
verb_test = verbs_1qisaa[1]
scroll = verb_test.to_scrolls.scroll[0]
ref = find_verb_ref(verb_test)
verse_control = TFOb.section(ref, BHSA)

print(find_verb_ref(verb_test))

print(f"BHSA verse: {verse_control}\n")

print("BHSA clauses:")
for clause in verse_control.to_clauses:
    print(clause)
    
print("\n")

print("BHSA phrases:")
for phrase in verse_control.to_phrases:
    if phrase.function[0] == "Cmpl" or phrase.function[0] == "Subj":
        print(phrase, phrase.function)
        
for word in verse_control.to_words:
    if word.uvf[0] == "H":
        print(f"Words with directive he in BHSA verse: {word}")
        
scroll = verb_test.to_scrolls.scroll[0]
print(f"\nDSS verse: {TFOb.section(ref, DSS, scroll)}")
print(f"DSS verb : {verb_test}")
print(f"DSS Clause: {find_clause(verb_test)}")
print(f"DSS Complement: {find_complements(verb_test)}")
print(f"DSS Subject: {find_subject(verb_test)}")

for word in TFOb.section(ref, DSS, scroll):
    if word.uvf_etcbc[0] == "H":
        print(f"Words with directive he in DSS verse: {word}")


['Isaiah', '57', '9']
BHSA verse: W TCRJ L MLK B CMN W TRBJ RQXJK W TCLXJ YRJK <D M RXQ W TCPJLJ <D C>WL

BHSA clauses:
W TCRJ L MLK B CMN
W TRBJ RQXJK
W TCLXJ YRJK <D M RXQ
W TCPJLJ <D C>WL


BHSA phrases:
L MLK ['Cmpl']
<D M RXQ ['Cmpl']
<D C>WL ['Cmpl']

DSS verse: W TCRJ L MLK B CMN W TRBJ RWQXJK W TCLXJ YJRJK <D M RXQ W TCPWLJ <D C>WL
DSS verb : TCRJ
DSS Clause: None
DSS Complement: None
DSS Subject: None


### Generate the Dataset

In [67]:
items = [] # create an empty list to store all the information for each occ.

       
for verb in chain(verbs_bhsa, verbs_1q8, verbs_1qisaa):
    
    # Add MT as "scroll" for the BHSA
    if verb.source.name == "BHSA":
        scroll = "MT"
        verse = verb.to_verses
        dir_he_dss_verse = ""
        sign_info = ""

    else:
        scroll = verb.to_scrolls.scroll[0]
        verse = TFOb.section([verb.book[0], verb.chapter[0], verb.verse[0]], DSS, scroll)
        dir_he_dss_verse = int("H" in verse.uvf_etcbc)
        sign_info = "".join([str(sign) for sign in is_sign_unc(verse)])
        
    subject = find_subject(verb)
    clause = find_clause(verb)
    complements = find_complements(verb)
    
    # If complements is None ==> there was no match between DSS and BHSA verses ==> find complement manually
    if complements is None:
        complements = [""]
        dir_he = ""

    for i, complement in enumerate(complements):        
        if complement == "":
            dir_he = ""
        else: 
            dir_he = int("H" in complement.to_words.uvf)
               
    
        # Collect information about the following variables:    
        item = {
            "verb_id": verb.ids[0], 
            "lex": verb.lex[0], 
            "scroll": scroll,
            "book": verb.book[0], 
            "chapter": verb.chapter[0], 
            "verse_num": verb.verse[0],
            "gcons_verb": clean(verb.g_cons[0]),
            "gcons_verse": clean(str(verse)),
            "gcons_clause": clean(str(clause)),
            "subject": clean(str(subject)),
            "complement": clean(str(complement)),
            "dir_he": dir_he,
            "dir_he_dss": dir_he_dss_verse,
            "sign_info": sign_info,
            "stem": verb.vs[0],
            "tense": verb.vt[0],
        }
        
        if complement != "":
            prepositions = find_prepositions(verb)[i]
            n = 0
            for preposition in prepositions:
                n += 1
                item[f"preposition_{n}"] = preposition.lex[0]

        items.append(item)

In [68]:
df = pd.DataFrame(items).fillna("")
df.sort_values(["book", "chapter", "verse_num"], ascending=[True, True, True], ignore_index=True, inplace=True)

In [70]:
df.head(25)

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,gcons_clause,subject,complement,dir_he,dir_he_dss,sign_info,stem,tense,preposition_1
0,220632,PF<[,MT,Isaiah,27,4,>PF<H,XMH >JN LJ MJ JTNNJ CMJR CJT B MLXMH >PF<H BH ...,B MLXMH >PF<H BH,,BH,0,,,qal,impf,B
1,1903759,PF<[,1Qisaa,Isaiah,27,4,>PF<H,XMH >JN LJ MJ JTNNJ CJMJR W CJT B MLXMH >PF<H ...,None,None,,,0,0000000000000000000000000000000000000000000000...,qal,impf,
2,231688,CWR=[,MT,Isaiah,57,9,TCRJ,W TCRJ L MLK B CMN W TRBJ RQXJK W TCLXJ YRJK <...,W TCRJ L MLK B CMN,,L MLK,0,,,qal,wayq,L
3,1915493,CWR=[,1Qisaa,Isaiah,57,9,TCRJ,W TCRJ L MLK B CMN W TRBJ RWQXJK W TCLXJ YJRJK...,None,None,,,0,0000000000000000000000000000000000000000000000...,qal,wayy,


In [71]:
df.to_csv("data/DATASET3.csv", index=False)

### WARNING: USE ONLY in case of need to merge datasets

In [72]:
df2 = pd.read_csv("data/MERGED_DATASET.csv").fillna("")

In [73]:
df2["dir_he"] = df2["dir_he"].astype(str)
df2["dir_he"] = df2["dir_he"].replace("1.0", "1").replace("0.0", "0")

df2["dir_he_dss"] = df2["dir_he_dss"].astype(str)
df2["dir_he_dss"] = df2["dir_he_dss"].replace("1.0", "1").replace("0.0", "0")

In [74]:
df2

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,gcons_clause,subject,complement,dir_he,dir_he_dss,sign_info,stem,tense,preposition_1,preposition_2,preposition_3,preposition_4
0,1895059,BW>[,1Qisaa,Isaiah,1,12,TB>W,KJ> TB>W L R>WT PNJ MJ BQC ZW>T M JDKM L RMWS ...,,,,,0,0000000000000000000000000000000000000000000,qal,impf,,,,
1,1895075,BW>[,1Qisaa,Isaiah,1,13,HBJ>,LW> TWSJPW L HBJ> MNXT CW> QVRT TW<BH HJ> LJ X...,,,,,0,0000000000000000000000000000000000000000000000...,hifil,infc,,,,
2,212315,SWR[,MT,Isaiah,1,16,HSJRW,RXYW HZKW HSJRW R< M<LLJKM M NGD <JNJ XDLW HR<,HSJRW R< M<LLJKM M NGD <JNJ,,M NGD <JNJ,,,,hif,impv,MN,,,
3,1895129,SWR[,1Qisaa,Isaiah,1,16,HSJRW,RXYW W HZKW W HSJRW RW< M<LLJKM M NGD <JNJ XDL...,W HSJRW RW< M<LLJKM M NGD <JNJ,,M NGD <JNJ,0,0,,hifil,impv,M,,,
4,1895149,HLK[,1Qisaa,Isaiah,1,18,LKW,LKW N> W NWKXH JW>MR JHWH >M JHJW XV>JKM K CNJ...,,,,,0,0000000000000000000000000000000000000000000000...,qal,impv,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
976,1898270,CWB[,1Qisaa,Isaiah,9,12,CB,W H <M LW> CB <L H MKHW W >T JHWH YB>WT LW> DRCW,,,,,0,00000000000000000000000000000000000000,qal,perf,,,,
977,1898352,CWB[,1Qisaa,Isaiah,9,16,CB,<L KN <L BXWRJW LW> JXMWL >DWNJ W >T JTWMJW W ...,,,,,0,0000000000000000000000000000000000000000000000...,qal,perf,,,,
978,1898435,CWB[,1Qisaa,Isaiah,9,20,CB,W J>KL MNCH >T >PRJM W >PRJM >T MNCH JXDW HMH ...,,,,,0,0000000000000000000000000000000000000000000000...,qal,perf,,,,
979,215297,NPL[,MT,Isaiah,9,7,NPL,DBR CLX >DNJ B J<QB W NPL B JFR>L,W NPL B JFR>L,,B JFR>L,,,,qal,perf,B,,,


In [75]:
dfs = [df, df2]
df3 = pd.concat(dfs)
df3

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,gcons_clause,subject,complement,dir_he,dir_he_dss,sign_info,stem,tense,preposition_1,preposition_2,preposition_3,preposition_4
0,220632,PF<[,MT,Isaiah,27,4,>PF<H,XMH >JN LJ MJ JTNNJ CMJR CJT B MLXMH >PF<H BH ...,B MLXMH >PF<H BH,,BH,0,,,qal,impf,B,NaN,NaN,NaN
1,1903759,PF<[,1Qisaa,Isaiah,27,4,>PF<H,XMH >JN LJ MJ JTNNJ CJMJR W CJT B MLXMH >PF<H ...,None,None,,,0,0000000000000000000000000000000000000000000000...,qal,impf,,NaN,NaN,NaN
2,231688,CWR=[,MT,Isaiah,57,9,TCRJ,W TCRJ L MLK B CMN W TRBJ RQXJK W TCLXJ YRJK <...,W TCRJ L MLK B CMN,,L MLK,0,,,qal,wayq,L,NaN,NaN,NaN
3,1915493,CWR=[,1Qisaa,Isaiah,57,9,TCRJ,W TCRJ L MLK B CMN W TRBJ RWQXJK W TCLXJ YJRJK...,None,None,,,0,0000000000000000000000000000000000000000000000...,qal,wayy,,NaN,NaN,NaN
0,1895059,BW>[,1Qisaa,Isaiah,1,12,TB>W,KJ> TB>W L R>WT PNJ MJ BQC ZW>T M JDKM L RMWS ...,,,,,0,0000000000000000000000000000000000000000000,qal,impf,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
976,1898270,CWB[,1Qisaa,Isaiah,9,12,CB,W H <M LW> CB <L H MKHW W >T JHWH YB>WT LW> DRCW,,,,,0,00000000000000000000000000000000000000,qal,perf,,,,
977,1898352,CWB[,1Qisaa,Isaiah,9,16,CB,<L KN <L BXWRJW LW> JXMWL >DWNJ W >T JTWMJW W ...,,,,,0,0000000000000000000000000000000000000000000000...,qal,perf,,,,
978,1898435,CWB[,1Qisaa,Isaiah,9,20,CB,W J>KL MNCH >T >PRJM W >PRJM >T MNCH JXDW HMH ...,,,,,0,0000000000000000000000000000000000000000000000...,qal,perf,,,,
979,215297,NPL[,MT,Isaiah,9,7,NPL,DBR CLX >DNJ B J<QB W NPL B JFR>L,W NPL B JFR>L,,B JFR>L,,,,qal,perf,B,,,


In [76]:
df3.drop(df3[(df3['lex']== "HWH[")].index, inplace=True)
df3.drop(df3[(df3['lex']== "PC<[")].index, inplace=True)
df3.drop(df3[(df3['lex']== "CWR[")].index, inplace=True)


df3.sort_values(["book", "chapter", "verse_num"], ascending=[True, True, True], ignore_index=True, inplace=True)
df3

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,gcons_clause,subject,complement,dir_he,dir_he_dss,sign_info,stem,tense,preposition_1,preposition_2,preposition_3,preposition_4
0,1895059,BW>[,1Qisaa,Isaiah,1,12,TB>W,KJ> TB>W L R>WT PNJ MJ BQC ZW>T M JDKM L RMWS ...,,,,,0,0000000000000000000000000000000000000000000,qal,impf,,,,
1,1895075,BW>[,1Qisaa,Isaiah,1,13,HBJ>,LW> TWSJPW L HBJ> MNXT CW> QVRT TW<BH HJ> LJ X...,,,,,0,0000000000000000000000000000000000000000000000...,hifil,infc,,,,
2,212315,SWR[,MT,Isaiah,1,16,HSJRW,RXYW HZKW HSJRW R< M<LLJKM M NGD <JNJ XDLW HR<,HSJRW R< M<LLJKM M NGD <JNJ,,M NGD <JNJ,,,,hif,impv,MN,,,
3,1895129,SWR[,1Qisaa,Isaiah,1,16,HSJRW,RXYW W HZKW W HSJRW RW< M<LLJKM M NGD <JNJ XDL...,W HSJRW RW< M<LLJKM M NGD <JNJ,,M NGD <JNJ,0,0,,hifil,impv,M,,,
4,1895149,HLK[,1Qisaa,Isaiah,1,18,LKW,LKW N> W NWKXH JW>MR JHWH >M JHJW XV>JKM K CNJ...,,,,,0,0000000000000000000000000000000000000000000000...,qal,impv,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
962,1918913,JY>[,1Qisaa,Isaiah,66,24,JY>W,W JY>W W R>W B PGRJ H >NCJM H PWC<JM BJ> KJ> T...,,,,,0,0000000000000000000000000000000000000000000000...,qal,perf,,,,
963,220632,PF<[,MT,Isaiah,27,4,>PF<H,XMH >JN LJ MJ JTNNJ CMJR CJT B MLXMH >PF<H BH ...,B MLXMH >PF<H BH,,BH,0,,,qal,impf,B,NaN,NaN,NaN
964,1903759,PF<[,1Qisaa,Isaiah,27,4,>PF<H,XMH >JN LJ MJ JTNNJ CJMJR W CJT B MLXMH >PF<H ...,None,None,,,0,0000000000000000000000000000000000000000000000...,qal,impf,,NaN,NaN,NaN
965,231688,CWR=[,MT,Isaiah,57,9,TCRJ,W TCRJ L MLK B CMN W TRBJ RQXJK W TCLXJ YRJK <...,W TCRJ L MLK B CMN,,L MLK,0,,,qal,wayq,L,NaN,NaN,NaN


In [77]:
df3[(df3.scroll == "MT")]

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,gcons_clause,subject,complement,dir_he,dir_he_dss,sign_info,stem,tense,preposition_1,preposition_2,preposition_3,preposition_4
2,212315,SWR[,MT,Isaiah,1,16,HSJRW,RXYW HZKW HSJRW R< M<LLJKM M NGD <JNJ XDLW HR<,HSJRW R< M<LLJKM M NGD <JNJ,,M NGD <JNJ,,,,hif,impv,MN,,,
5,212418,BW>[,MT,Isaiah,1,23,JBW>,FRJK SWRRJM W XBRJ GNBJM KLW >HB CXD W RDP CLM...,W RJB >LMNH L> JBW> >LJHM,RJB >LMNH,>LJHM,,,,qal,impf,>L,,,
7,212437,CWB[,MT,Isaiah,1,25,>CJBH,W >CJBH JDJ <LJK W >YRP K BR SJGJK W >SJRH KL ...,W >CJBH JDJ <LJK,,<LJK,,,,hif,impf,<L,,,
11,212564,NHR[,MT,Isaiah,2,2,NHRW,W HJH B >XRJT H JMJM NKWN JHJH HR BJT JHWH B R...,W NHRW >LJW KL H GWJM,KL H GWJM,>LJW,,,,qal,perf,>L,,,
13,212577,<LH[,MT,Isaiah,2,3,N<LH,W HLKW <MJM RBJM W >MRW LKW W N<LH >L HR JHWH ...,W N<LH >L HR JHWH >L BJT >LHJ J<QB,,>L HR JHWH >L BJT >LHJ J<QB,,,,qal,impf,>L,>L,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
941,234608,BW>[,MT,Isaiah,66,7,JBW>,B VRM TXJL JLDH B VRM JBW> XBL LH W HMLJVH ZKR,B VRM JBW> XBL LH,XBL,LH,,,,qal,impf,L,,,
953,234871,BW>[,MT,Isaiah,66,20,HBJ>W,W HBJ>W >T KL >XJKM M KL H GWJM MNXH L JHWH B ...,W HBJ>W >T KL >XJKM M KL H GWJM MNXH L JHWH B ...,,<L HR QDCJ JRWCLM,,,,hif,perf,<L,,,
954,234909,BW>[,MT,Isaiah,66,20,JBJ>W,W HBJ>W >T KL >XJKM M KL H GWJM MNXH L JHWH B ...,K >CR JBJ>W BNJ JFR>L >T H MNXH B KLJ VHWR BJT...,BNJ JFR>L,BJT JHWH,,,,hif,impf,,,,
963,220632,PF<[,MT,Isaiah,27,4,>PF<H,XMH >JN LJ MJ JTNNJ CMJR CJT B MLXMH >PF<H BH ...,B MLXMH >PF<H BH,,BH,0,,,qal,impf,B,NaN,NaN,NaN


In [78]:
df3.to_csv("data/final_isaiah_dataset.csv", index=False)